In [ ]:
from statsmodels.stats.power import TTestPower 

In [2]:
 
from statsmodels.stats.power import TTestPower 
  
  
power = TTestPower() 
n_test = power.solve_power(nobs=8, effect_size = 0.5, 
                           power = None, alpha = 0.05) 
print('Power: {:.3f}'.format(n_test)) 

Power: 0.232


In [4]:
import numpy as np
import pandas as pd
from statsmodels.regression.mixed_linear_model import MixedLM
from tqdm import tqdm

def simulate_power(n_mice=8, n_neurons=100, effect_size=0.2, 
                  icc=0.3, n_simulations=1000, alpha=0.05):
    """
    Simulate power for calcium imaging experiment with:
    - n_mice: number of mice
    - n_neurons: neurons per mouse
    - effect_size: expected effect size (as proportion)
    - icc: intraclass correlation coefficient
    - n_simulations: number of simulations to run
    - alpha: significance level
    """
    significant_results = 0
    
    for _ in tqdm(range(n_simulations)):
        # Create empty dataframe
        data = []
        
        # Generate data for each mouse
        for mouse_id in range(n_mice):
            # Half mice in control, half in treatment
            condition = 'treatment' if mouse_id < n_mice/2 else 'control'
            
            # Mouse-specific random effect
            mouse_effect = np.random.normal(0, np.sqrt(icc))
            
            # Generate neurons for this mouse
            for neuron in range(n_neurons):
                # Fixed effect of treatment
                fixed_effect = effect_size if condition == 'treatment' else 0
                
                # Neuron-specific random effect
                neuron_effect = np.random.normal(0, np.sqrt(1-icc))
                
                # Response variable (e.g., ΔF/F)
                response = fixed_effect + mouse_effect + neuron_effect
                
                data.append({
                    'mouse_id': mouse_id,
                    'neuron_id': neuron,
                    'condition': condition,
                    'response': response
                })
        
        # Convert to dataframe
        df = pd.DataFrame(data)
        
        # Fit mixed-effects model
        model = MixedLM.from_formula(
            "response ~ condition", 
            groups="mouse_id", 
            data=df
        )
        result = model.fit()
        
        # Check if treatment effect is significant
        p_value = result.pvalues['condition[T.treatment]']
        if p_value < alpha:
            significant_results += 1
    
    # Calculate power
    power = significant_results / n_simulations
    return power

# Run simulation with different numbers of mice
mice_range = [5, 8, 10, 12, 15]
powers = []

for n_mice in mice_range:
    power = simulate_power(n_mice=n_mice, n_neurons=1200, effect_size=0.2, icc=0.3)
    powers.append(power)
    print(f"Power with {n_mice} mice: {power:.3f}")

  4%|▎         | 36/1000 [00:03<01:40,  9.62it/s]C:\Users\Conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
  8%|▊         | 83/1000 [00:09<01:41,  9.06it/s]C:\Users\Conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to 

Power with 5 mice: 0.172


  2%|▏         | 18/1000 [00:03<02:51,  5.73it/s]


KeyboardInterrupt: 

In [5]:
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm

def simulate_power_aggregated(n_mice=8, n_neurons=100, effect_size=0.2, 
                             icc=0.3, n_simulations=1000, alpha=0.05):
    """
    Simulate power for calcium imaging experiment using aggregated data at mouse level.
    """
    significant_results = 0
    
    for _ in tqdm(range(n_simulations)):
        # Generate mouse-level means
        treatment_mice = []
        control_mice = []
        
        # Generate data for each mouse
        for mouse_id in range(n_mice):
            # Half mice in control, half in treatment
            is_treatment = mouse_id < n_mice/2
            
            # Mouse-specific random effect
            mouse_effect = np.random.normal(0, np.sqrt(icc))
            
            # Generate neurons for this mouse and calculate mean
            neuron_responses = []
            for _ in range(n_neurons):
                # Fixed effect of treatment
                fixed_effect = effect_size if is_treatment else 0
                
                # Neuron-specific random effect
                neuron_effect = np.random.normal(0, np.sqrt(1-icc))
                
                # Response variable (e.g., ΔF/F)
                response = fixed_effect + mouse_effect + neuron_effect
                neuron_responses.append(response)
            
            # Calculate mean response for this mouse
            mouse_mean = np.mean(neuron_responses)
            
            if is_treatment:
                treatment_mice.append(mouse_mean)
            else:
                control_mice.append(mouse_mean)
        
        # Run t-test on mouse-level data
        t_stat, p_value = stats.ttest_ind(treatment_mice, control_mice)
        
        if p_value < alpha:
            significant_results += 1
    
    # Calculate power
    power = significant_results / n_simulations
    return power

# Run simulation with different numbers of mice
mice_range = [5, 8, 10, 12, 15]
powers = []

for n_mice in mice_range:
    power = simulate_power_aggregated(n_mice=n_mice, n_neurons=100, effect_size=0.2, icc=0.3, n_simulations=1000)
    powers.append(power)
    print(f"Power with {n_mice} mice: {power:.3f}")

100%|██████████| 1000/1000 [00:03<00:00, 266.85it/s]


Power with 5 mice: 0.069


100%|██████████| 1000/1000 [00:04<00:00, 245.66it/s]


Power with 8 mice: 0.076


100%|██████████| 1000/1000 [00:04<00:00, 200.02it/s]


Power with 10 mice: 0.075


100%|██████████| 1000/1000 [00:05<00:00, 196.89it/s]


Power with 12 mice: 0.083


100%|██████████| 1000/1000 [00:05<00:00, 175.26it/s]

Power with 15 mice: 0.102


In [6]:
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm

def simulate_power_with_real_params(n_mice=8, n_neurons=300, 
                                   mean_diff_percent=20, 
                                   control_mean=1.0,  # Baseline activity level
                                   between_mouse_cv=0.3,  # Coefficient of variation between mice
                                   within_mouse_cv=0.5,  # Coefficient of variation within mice
                                   n_simulations=1000, alpha=0.05):
    """
    Simulate power using parameters matched to real calcium imaging data.
    - mean_diff_percent: Percent change between conditions (20% = 0.2)
    - control_mean: Baseline average response
    - between_mouse_cv: Coefficient of variation between mice (mouse-to-mouse variability)
    - within_mouse_cv: Coefficient of variation within mouse (neuron-to-neuron variability)
    """
    significant_results = 0
    
    # Convert percent difference to absolute
    treatment_mean = control_mean * (1 + mean_diff_percent/100)
    
    # Calculate standard deviations
    between_mouse_sd = control_mean * between_mouse_cv
    within_mouse_sd = control_mean * within_mouse_cv
    
    for _ in tqdm(range(n_simulations)):
        # Generate mouse-level means
        treatment_mice = []
        control_mice = []
        
        # Generate data for each mouse
        n_treatment = n_mice // 2
        n_control = n_mice - n_treatment
        
        # Treatment mice
        for _ in range(n_treatment):
            # Mouse-specific baseline (random effect)
            mouse_baseline = np.random.normal(treatment_mean, between_mouse_sd)
            
            # Generate neurons for this mouse
            neuron_responses = np.random.normal(mouse_baseline, within_mouse_sd, size=n_neurons)
            mouse_mean = np.mean(neuron_responses)
            treatment_mice.append(mouse_mean)
        
        # Control mice
        for _ in range(n_control):
            # Mouse-specific baseline (random effect)
            mouse_baseline = np.random.normal(control_mean, between_mouse_sd)
            
            # Generate neurons for this mouse
            neuron_responses = np.random.normal(mouse_baseline, within_mouse_sd, size=n_neurons)
            mouse_mean = np.mean(neuron_responses)
            control_mice.append(mouse_mean)
        
        # Run t-test on mouse-level data
        t_stat, p_value = stats.ttest_ind(treatment_mice, control_mice)
        
        if p_value < alpha:
            significant_results += 1
    
    # Calculate power
    power = significant_results / n_simulations
    return power

# Test different parameter combinations to match your observed results
parameter_sets = [
    {"between_mouse_cv": 0.15, "within_mouse_cv": 0.30},  # Low variability between mice
    {"between_mouse_cv": 0.20, "within_mouse_cv": 0.40},  # Medium variability
    {"between_mouse_cv": 0.25, "within_mouse_cv": 0.50}   # Higher variability
]

# Run simulations
for params in parameter_sets:
    power = simulate_power_with_real_params(
        n_mice=8, 
        n_neurons=300,  # Assuming 200 neurons per mouse 
        mean_diff_percent=20,  # 20% difference
        between_mouse_cv=params["between_mouse_cv"],
        within_mouse_cv=params["within_mouse_cv"],
        n_simulations=1000
    )
    print(f"Power with parameters {params}: {power:.3f}")

100%|██████████| 1000/1000 [00:02<00:00, 489.70it/s]


Power with parameters {'between_mouse_cv': 0.15, 'within_mouse_cv': 0.3}: 0.368


100%|██████████| 1000/1000 [00:02<00:00, 482.25it/s]


Power with parameters {'between_mouse_cv': 0.2, 'within_mouse_cv': 0.4}: 0.207


100%|██████████| 1000/1000 [00:02<00:00, 449.55it/s]

Power with parameters {'between_mouse_cv': 0.25, 'within_mouse_cv': 0.5}: 0.147
